<a href="https://colab.research.google.com/github/MitalPattani/M5-Forecasting-Accuracy/blob/master/sales_prediction_accuracy(data_visualization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install stationarizer

In [3]:
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from stationarizer import simple_auto_stationarize
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [4]:
calendar = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/calendar.csv")
sales_data = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/sales_train_validation.csv")
sell_price = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/sell_prices.csv")

In [30]:
sales_data.head

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [20]:
calendar.info()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
sales_data['cat_id'].value_counts()

FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: cat_id, dtype: int64

In [25]:
level_groupings = {0: ["state_id"], 1: ["store_id"], 2: ["cat_id"], 3: ["dept_id"], 
              4: ["state_id", "cat_id"], 5: ["state_id", "dept_id"], 6: ["store_id", "cat_id"], 7: ["store_id", "dept_id"],
              8: ["item_id"], 9: ["item_id", "state_id"]}

In [5]:
cat_sales = sales_data.copy()
cat_sales = cat_sales.groupby(by=['state_id','cat_id'], as_index = False).sum()
cat_sales['state_cat'] = cat_sales['state_id']+'_'+cat_sales['cat_id']
cat_sales = cat_sales.drop(['state_id','cat_id'],axis=1)
cat_sales = cat_sales.T
cat_sales.columns = cat_sales.loc['state_cat']
cat_sales = cat_sales.drop('state_cat')

In [15]:
cat_sales_holiday = pd.merge(cat_sales,calendar, left_index = True, right_on = 'd')
cat_sales_holiday = cat_sales_holiday.drop(['date','wm_yr_wk','weekday','wday','month','year','snap_CA','snap_TX','snap_WI'], axis = 1)
cat_sales_holiday.at[cat_sales_holiday['event_name_1'].notna() | cat_sales_holiday['event_name_1'].notna(),'is_holiday'] = 1
cat_sales_holiday.at[cat_sales_holiday['event_name_1'].isna() & cat_sales_holiday['event_name_1'].isna(),'is_holiday'] = 0
cat_sales_holiday['is_holiday'] = cat_sales_holiday['is_holiday'].astype(int)
cat_sales_holiday['d'] = cat_sales_holiday.d.str.lstrip('d_')


In [6]:
cat_sales_holiday.head()

,CA_FOODS,CA_HOBBIES,CA_HOUSEHOLD,TX_FOODS,TX_HOBBIES,TX_HOUSEHOLD,WI_FOODS,WI_HOBBIES,WI_HOUSEHOLD,d,event_name_1,event_type_1,event_name_2,event_type_2,is_holiday
0,10101,1802,2292,6853,879,1706,6224,1083,1691,1,NaN,NaN,NaN,NaN,0
1,9862,1561,2382,7030,870,1730,5866,926,1522,2,NaN,NaN,NaN,NaN,0
2,6944,1472,1692,5124,526,1128,5106,684,1107,3,NaN,NaN,NaN,NaN,0
3,7864,1405,1778,5470,809,1102,5544,455,985,4,NaN,NaN,NaN,NaN,0
4,7178,1181,1566,4602,501,809,2823,132,354,5,NaN,NaN,NaN,NaN,0


In [16]:
CA_FOODS = pd.DataFrame({'Data':cat_sales_holiday['CA_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# CA_FOODS['Data'] = CA_FOODS['Data'].apply(lambda x: np.log(x))
CA_FOODS = CA_FOODS[CA_FOODS['Data'] > 3]

CA_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['CA_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# CA_HOBBIES = CA_HOBBIES[CA_HOBBIES['Data'] > 0]

CA_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['CA_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# CA_HOUSEHOLD= CA_HOUSEHOLD[CA_HOUSEHOLD['Data'] > 0]

TX_FOODS = pd.DataFrame({'Data':cat_sales_holiday['TX_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# TX_FOODS = TX_FOODS[TX_FOODS['Data'] > 0.5]
# TX_FOODS = TX_FOODS[TX_FOODS['Data'] < 3.0]

TX_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['TX_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# TX_HOBBIES = TX_HOBBIES[TX_HOBBIES['Data'] > 0]

TX_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['TX_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# TX_HOUSEHOLD = TX_HOUSEHOLD[TX_HOUSEHOLD['Data'] > 0] 

WI_FOODS = pd.DataFrame({'Data':cat_sales_holiday['WI_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# WI_FOODS = WI_FOODS[WI_FOODS['Data'] > 0.5]

WI_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['WI_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# WI_HOBBIES = WI_HOBBIES[WI_HOBBIES['Data'] > 0]

WI_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['WI_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
# WI_HOUSEHOLD = WI_HOUSEHOLD[WI_HOUSEHOLD['Data'] > 0.05]

categories = [CA_FOODS, CA_HOBBIES, CA_HOUSEHOLD, TX_FOODS, TX_HOBBIES, TX_HOUSEHOLD, WI_FOODS, WI_HOBBIES, WI_HOUSEHOLD]
categories_name = ('CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD')


In [64]:
fig = make_subplots(rows=3, cols=3, subplot_titles = categories_name)
i = 1
j = 1
for ind, val in enumerate(categories):
    if j > 3:
        j = 1
        i = i+1
    fig.add_trace(
        go.Scatter(x= val['d'], y=val['Data']),
        row=i, col=j
    )
    j = j+1

fig.update_layout(height=600, width=800, title_text="Total mean sales")
fig.show()


In [37]:
holi_sales = sales_data.copy()
holi_sales = holi_sales.drop(columns=['id','item_id','dept_id','store_id','state_id'])
holi_sales = holi_sales.groupby(by=['cat_id']).sum()
holi_sales = holi_sales.T

In [50]:
cat_holiday = pd.merge(holi_sales,calendar, left_index = True, right_on = 'd')
cat_holiday = cat_holiday.drop(['date','wm_yr_wk','weekday','wday','month','snap_CA','snap_TX','snap_WI'], axis = 1)
cat_holiday.at[cat_holiday['event_name_1'].notna() | cat_holiday['event_name_1'].notna(),'is_holiday'] = 1
cat_holiday.at[cat_holiday['event_name_1'].isna() & cat_holiday['event_name_1'].isna(),'is_holiday'] = 0
cat_holiday['is_holiday'] = cat_holiday['is_holiday'].astype(int)
cat_holiday['d'] = cat_holiday.d.str.lstrip('d_')
cat_holiday_2011 = cat_holiday[cat_holiday['year']==2011]
cat_holiday_2012 = cat_holiday[cat_holiday['year']==2012].reset_index(drop = True)
cat_holiday_2013 = cat_holiday[cat_holiday['year']==2013].reset_index(drop = True)
cat_holiday_2014 = cat_holiday[cat_holiday['year']==2014].reset_index(drop = True)
cat_holiday_2015 = cat_holiday[cat_holiday['year']==2015].reset_index(drop = True)
cat_holiday_2016 = cat_holiday[cat_holiday['year']==2016].reset_index(drop = True)


,FOODS,HOBBIES,HOUSEHOLD,year,d,event_name_1,event_type_1,event_name_2,event_type_2,is_holiday
0,15898,1825,4084,2012,338,NewYear,National,NaN,NaN,1
1,23631,3082,6340,2012,339,NaN,NaN,NaN,NaN,0
2,21115,2649,4987,2012,340,NaN,NaN,NaN,NaN,0
3,18627,2526,4668,2012,341,NaN,NaN,NaN,NaN,0
4,21132,2433,4934,2012,342,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
361,19236,2183,7667,2012,699,NaN,NaN,NaN,NaN,0
362,21208,2377,8762,2012,700,NaN,NaN,NaN,NaN,0
363,24343,2555,10256,2012,701,NaN,NaN,NaN,NaN,0
364,24322,2352,9025,2012,702,NaN,NaN,NaN,NaN,0


In [53]:
years = ['2011','2012','2013','2014','2015','2016']
plots = [cat_holiday_2011,cat_holiday_2012,cat_holiday_2013,cat_holiday_2014,cat_holiday_2015,cat_holiday_2016]
fig = make_subplots(rows=3, cols=2, subplot_titles = years)
i = 1
j = 1
for ind, val in enumerate(plots):
    if j > 2:
        j = 1
        i = i+1
    fig.add_trace(
        go.Scatter(x= val.index, y=val['FOODS'], name='Foods'),
        row=i, col=j
    )
    fig.add_trace(
        go.Scatter(x= val.index, y=val['HOBBIES'], name='Hobbies'),
        row=i, col=j
    )
    fig.add_trace(
        go.Scatter(x= val.index, y=val['HOUSEHOLD'], name='Household'),
        row=i, col=j
    )
    j = j+1

fig.update_layout(title_text="categorical sales per year")
fig.show()


In [ ]:
# We can see that Every year foods categories have the highest sales followed by house and the hobbies.
# next we will try to plot major holidays for each category.

In [71]:
holiday_sales = cat_holiday[cat_holiday['is_holiday']==1]
household = holiday_sales.nlargest(10, 'HOUSEHOLD')
foods = holiday_sales.nlargest(10, 'FOODS')
hobbies = holiday_sales.nlargest(10, 'HOBBIES')
total = pd.concat([foods,hobbies,household]).drop_duplicates()
total

,FOODS,HOBBIES,HOUSEHOLD,year,d,event_name_1,event_type_1,event_name_2,event_type_2,is_holiday
947,34391,2580,10101,2013,948,LaborDay,National,NaN,NaN,1
1842,32998,4103,11236,2016,1843,ValentinesDay,Cultural,NaN,NaN,1
1835,32421,4086,10386,2016,1836,SuperBowl,Sporting,NaN,NaN,1
583,32349,2469,9707,2012,584,LaborDay,National,NaN,NaN,1
1884,32236,3643,10481,2016,1885,Easter,Cultural,NaN,NaN,1
1843,31682,3593,10877,2016,1844,PresidentsDay,National,NaN,NaN,1
1344,31579,4251,11369,2014,1345,EidAlAdha,Religious,NaN,NaN,1
652,31469,3004,9044,2012,653,VeteransDay,National,NaN,NaN,1
1100,31405,3506,8986,2014,1101,SuperBowl,Sporting,NaN,NaN,1
1534,31177,4156,11847,2015,1535,OrthodoxEaster,Religious,NaN,NaN,1


In [70]:
fig = go.Figure()
fig.add_trace(
        go.Scatter(x= total['event_name_1'], y=total['FOODS'] ,mode='markers', name='Foods'),
)
fig.add_trace(
    go.Scatter(x= total['event_name_1'], y=total['HOBBIES'],mode='markers', name='Hobbies'),
)
fig.add_trace(
    go.Scatter(x= total['event_name_1'], y=total['HOUSEHOLD'],mode='markers', name='Household'),
)
fig.show()

In [30]:
one  = sales_data.iloc[4]
one = one.drop(['id','item_id','dept_id','cat_id','store_id','state_id'])
fig = make_subplots(rows=1)
fig.add_trace(go.Scatter(x= list(range(1,1913)), y=one))
fig.show()

In [27]:
## Random forest with mixed effect
sales_melt = sales_data.copy()
sales_melt = sales_melt.melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'])
sales_melt.rename(columns={"value": "sales"})
sales_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   id        object
 1   item_id   object
 2   dept_id   object
 3   cat_id    object
 4   store_id  object
 5   state_id  object
 6   variable  object
 7   value     int64 
dtypes: int64(1), object(7)
memory usage: 3.5+ GB
